In [1]:
import requests,zipfile, io
from pathlib import Path
from os import walk,utime
import glob2
import time

In [2]:
cookies = {"gclubsess":"4df45542b47d394aad4caf33b5111c7d8cefbb2f"}
player_list=[767208,767166,765902,866021,1284651,759993,109631,47742,273342,593990,559439]
year=2021
month=4
dest_folder = "/home/marcel/projetos/data/csgo_analyze/replays/abr/"

In [3]:
def get_player_matches(player_id,year,month):
    month_string = str(year)+"-"+str(month).zfill(2)
    history_url="https://gamersclub.com.br/api/box/historyFilterDate/"+str(player_id)+"/"+month_string
    r = requests.get(history_url, cookies=cookies)
    gc_history_data=r.json()
    
    total_matches=gc_history_data["matches"]["matches"]
    matches={}
    if total_matches > 0:
        matches_list=gc_history_data["monthMatches"]
        for match in matches_list:
                matches[match["id"]]=match
                matches[match["id"]]["partipating_players"]=1

        page_count=0
        while len(matches)<total_matches:
            page_count+=1
            history_page_url="https://gamersclub.com.br/api/box/historyMatchesPage/" \
                +str(player_id)+"/"+month_string+"/"+str(page_count)
            r = requests.get(history_page_url, cookies=cookies)

            for match in r.json()["monthMatches"]:
                if not (match["id"] in matches):
                    matches[match["id"]]=match
                    matches[match["id"]]["partipating_players"]=1

    return matches

In [4]:
matches_dicts=[]
for player_id in player_list:
    
    matches_dicts.append(get_player_matches(player_id,year,month))

    
merged_dict={}
for d in matches_dicts:
    for k,v in d.items():
        if k in merged_dict:
            merged_dict[k]["partipating_players"]+=1
        else:
            merged_dict[k]=v
        

merged_dict

{11429239: {'id': 11429239,
  'teamNameA': 'PHIL',
  'scoreA': 16,
  'teamNameB': 'Tnk',
  'scoreB': 8,
  'win': 0,
  'ratingPlayer': 1489,
  'ratingDiff': -17,
  'map': 'de_overpass',
  'type': 'lobby',
  'partipating_players': 5},
 11440217: {'id': 11440217,
  'teamNameA': 'C4 Base',
  'scoreA': 22,
  'teamNameB': 'Mega',
  'scoreB': 18,
  'win': 1,
  'ratingPlayer': 1508,
  'ratingDiff': 19,
  'map': 'de_train',
  'type': 'lobby',
  'partipating_players': 5},
 11442801: {'id': 11442801,
  'teamNameA': '=_=',
  'scoreA': 16,
  'teamNameB': 'C4 Base',
  'scoreB': 8,
  'win': 0,
  'ratingPlayer': 1494,
  'ratingDiff': -14,
  'map': 'de_inferno',
  'type': 'lobby',
  'partipating_players': 5},
 11444043: {'id': 11444043,
  'teamNameA': 'C4 Base',
  'scoreA': 9,
  'teamNameB': 'CABECA DE MELANCIA',
  'scoreB': 16,
  'win': 0,
  'ratingPlayer': 1478,
  'ratingDiff': -16,
  'map': 'de_dust2',
  'type': 'lobby',
  'partipating_players': 5},
 11445324: {'id': 11445324,
  'teamNameA': 'C4 Bas

In [5]:
def filter_matches_to_download(all_matches):
    filtered_matches={}
    for i, (match_id, match) in enumerate(all_matches.items()):
        reason=""
        if not match["partipating_players"]>=3:
            reason = "It's only "+str(match["partipating_players"])+ " participating players."

        if len(glob2.glob(dest_folder+"*_"+str(match_id)+"_*"))>0:
            reason= "It's been downloaded before."

        if reason=="":
            get_download_url="https://gamersclub.com.br/lobby/demoDownload/"+str(match_id)
            r = requests.get(get_download_url, cookies=cookies)
            response_data = r.json()
            if not response_data["success"]:
                print("Error",i,match,response_data)

            else: 
                filtered_matches[match_id]=match
                filtered_matches[match_id]["replay_url"]=response_data["demo"]
                
        else:        
            print("Removing match",match_id,"-",reason)
    return filtered_matches

In [6]:
filtered_matches=filter_matches_to_download(merged_dict)
filtered_matches

Removing match 11429239 - It's been downloaded before.
Removing match 11440217 - It's been downloaded before.
Removing match 11442801 - It's been downloaded before.
Removing match 11444043 - It's been downloaded before.
Removing match 11445324 - It's been downloaded before.
Removing match 11446282 - It's been downloaded before.
Removing match 11452755 - It's been downloaded before.
Removing match 11453634 - It's been downloaded before.
Removing match 11461108 - It's been downloaded before.
Removing match 11462190 - It's been downloaded before.
Removing match 11463059 - It's been downloaded before.
Removing match 11465359 - It's been downloaded before.
Removing match 11466010 - It's been downloaded before.
Removing match 11620845 - It's been downloaded before.
Removing match 11622392 - It's been downloaded before.
Removing match 11624167 - It's been downloaded before.
Removing match 11625455 - It's been downloaded before.
Removing match 11632955 - It's been downloaded before.
Removing m

{11806396: {'id': 11806396,
  'teamNameA': 'C4 Base',
  'scoreA': 16,
  'teamNameB': 'THE GORDO',
  'scoreB': 10,
  'win': 1,
  'ratingPlayer': 1507,
  'ratingDiff': 14,
  'map': 'de_nuke',
  'type': 'lobby',
  'partipating_players': 5,
  'replay_url': 'https://prod-demo-parser-gc-demos.s3.amazonaws.com/2021-04-28__0223__1__11806396__de_nuke__timethegordo__vs__c4base.zip'},
 11816708: {'id': 11816708,
  'teamNameA': 'MrS',
  'scoreA': 7,
  'teamNameB': 'Minfres Guves',
  'scoreB': 16,
  'win': 0,
  'ratingPlayer': 2063,
  'ratingDiff': -15,
  'map': 'de_vertigo',
  'type': 'lobby',
  'partipating_players': 4,
  'replay_url': 'https://prod-demo-parser-gc-demos.s3.amazonaws.com/2021-04-28__2111__1__11816708__de_vertigo__timeminfresguves__vs__timemrs.zip'},
 11829691: {'id': 11829691,
  'teamNameA': 'MrS',
  'scoreA': 16,
  'teamNameB': 'Tnk',
  'scoreB': 13,
  'win': 0,
  'ratingPlayer': 2052,
  'ratingDiff': -11,
  'map': 'de_train',
  'type': 'lobby',
  'partipating_players': 7,
  'rep

In [7]:
len(filtered_matches)

6

In [8]:
def extract_all_keep_date(z,match_file):
    for f in z.infolist():
        name, date_time = f.filename, f.date_time
        with open(match_file, 'wb') as outFile:
            outFile.write(z.open(f).read())
        date_time = time.mktime(date_time + (0, 0, -1))
        utime(match_file, (date_time, date_time))

In [9]:
def download_replays(matches):
    for _,match in matches.items():
        direct_download_url=match["replay_url"]
        match_name=direct_download_url[direct_download_url.rfind("/")+1:-4]
        match_file = Path(dest_folder+match_name+".dem")
        if not match_file.is_file():
            r = requests.get(direct_download_url)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            extract_all_keep_date(z,match_file)
        else:
            print(match_name, "was downloaded before.")
    

In [10]:
download_replays(filtered_matches)